In [1]:
%run stdPackages.ipynb
from vectorized import *
self = infHorizon(ni = 5, T = 10)

The infinite horizon PEE model consists of 7 equations for each $t$; all 7 equations are only forward-looking in that they specifically depend on the future policy $\tau_{t+1}$. For the infinite horizon model this means that the 7 equations are implemented for all $t<T$. For $t=T$ the same set of equations is used, albeit with the condition $\tau_{t+1}=\tau_t$.

Given this solution, we can outline the rest of the solution including individual savings rates, political support etc.

In [2]:
candidateVector = pd.Series(range(len(self.mainIndex)), index = self.mainIndex)
x = candidateVector.values

We get the correct elements from the $x$ vector using the 'get' method. This extracts on symbol name + lags in the vector:

In [3]:
τ  = self.get(x, 'τ') # from a vector of all the values, extract τ
τp = self.get(x, 'τ', l = -1) # extract τ but lagged -1 (i.e. vector of τ_{t+1})

This solves the economic equilibrium given policies:

In [4]:
τ  = np.full((self.T,), .1)
τp = np.full((self.T,), .1) 
solEE = self.solveCoreEE(τ, τp, x0 = None)

This solves for auxiliary log-derivatives:

In [5]:
solLnDev = self.solveLnDevs(self.getEE(solEE.values, 'Θh'),
                            self.getEE(solEE.values, 'Θs'),
                            self.getEE(solEE.values, 'Υ'),
                            τ, τp, x0 = None)

This solves the entire thing in one go:

In [6]:
solPEE = self.solveCorePEE()
solPEE

t  variable
0  dlnΘh      -0.245108
   dlnΘs      -1.225540
   dlnΥ       -1.225540
   Θh          1.328966
   Θs          0.094205
                 ...   
9  dlnΥ       -1.225540
   Θh          1.328966
   Θs          0.094205
   Υ           0.314797
   τ           0.093370
Length: 70, dtype: float64

Extract solution and compute coefficients/variables:

In [7]:
self.unloadCorePEE(solPEE)
self.reportCoefficients() # Get all the Θ coefficients
self.reportLevels() # Get the levels of savings, consumption, and labor supply
self.reportUtils() # Get level of utility for various consumers